In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import polars as pl
from make_clinical_dataset.epic.util import plot_count_over_time
from make_clinical_dataset.shared.constants import ROOT_DIR

pl.Config.set_fmt_str_lengths(100)
pl.Config.set_tbl_rows(200)

# Processed

In [ ]:
DATE = '2025-01-08'
NOTES_PATH = f'{ROOT_DIR}/data/processed/clinical_notes/data_pull_{DATE}/merged_processed_cleaned_clinical_notes.parquet.gzip'

In [ ]:
notes = pl.scan_parquet(NOTES_PATH)
notes = notes.rename({'Observations.ProcName': 'proc_name'})
notes = notes.with_columns(pl.col('processed_date').dt.year().alias('year'))
notes = notes.filter(pl.col('year') >= 2000) # remove notes older than 2000

In [ ]:
notes.group_by(['EPIC_FLAG', 'proc_name']).len().sort('EPIC_FLAG', 'len', descending=True).collect()

In [ ]:
counts = notes.group_by('proc_name', 'year').agg(pl.len()).sort('len', descending=True).collect()
counts = counts.to_pandas()
plot_count_over_time(counts, x='year', y='len', catcol='proc_name')